In [1]:
import os
import sys

current_dir = os.getcwd()

parent_dir = os.path.abspath(os.path.join(current_dir, ".."))

if parent_dir not in sys.path:
    sys.path.append(parent_dir)

In [2]:
from treemort.main import run
from treemort.utils.config import setup

config_file_path = "../configs/detr_bs8_cs256.txt"

conf = setup(config_file_path)

# Modified Config Variables for Local Execution
conf.data_folder = "/Users/anisr/Documents/AerialImageModel_ITD"
conf.output_dir = os.path.join("..", conf.output_dir)

eval_only = True

run(conf, eval_only)

/Users/anisr/Documents/TreeSeg/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[INFO] displaying the DETR configuration...


Some weights of DetrModel were not initialized from the model checkpoint at facebook/detr-resnet-50-panoptic and are newly initialized: ['backbone.conv_encoder.model.bn1.bias', 'backbone.conv_encoder.model.bn1.running_mean', 'backbone.conv_encoder.model.bn1.running_var', 'backbone.conv_encoder.model.bn1.weight', 'backbone.conv_encoder.model.conv1.weight', 'backbone.conv_encoder.model.layer1.0.bn1.bias', 'backbone.conv_encoder.model.layer1.0.bn1.running_mean', 'backbone.conv_encoder.model.layer1.0.bn1.running_var', 'backbone.conv_encoder.model.layer1.0.bn1.weight', 'backbone.conv_encoder.model.layer1.0.bn2.bias', 'backbone.conv_encoder.model.layer1.0.bn2.running_mean', 'backbone.conv_encoder.model.layer1.0.bn2.running_var', 'backbone.conv_encoder.model.layer1.0.bn2.weight', 'backbone.conv_encoder.model.layer1.0.bn3.bias', 'backbone.conv_encoder.model.layer1.0.bn3.running_mean', 'backbone.conv_encoder.model.layer1.0.bn3.running_var', 'backbone.conv_encoder.model.layer1.0.bn3.weight', 'ba

[INFO] Custom model created and weights loaded successfully.
No checkpoint found. Proceeding without loading weights.
Evaluation-only mode started.


Evaluating:   2%|▏         | 1/51 [00:08<07:24,  8.88s/it, iterations_left=50]